In a previous notebook, we sought the ground state of many-body systems, and connected this problem to binary optimization computer science. As we have seen, this connection is useful to understand for adiabatic quantum computing and variational algorithms.

However, we also talked about the engineering constraints we face in real devices: the evolution in these devices is actually that of an open quantum system, where the quantum processing unit interacts with the environment.

<img src="figures/open_system.svg" alt="A quantum processor as an open quantum system" style="width: 400px;"/>

The environment has a defined temperature and if we let the system equilibrate, the QPU becomes thermalized and the energy of the states will follow a Boltzmann distribution.

When using quantum computers for optimization, we retain one sample out of the hundreds or thousands, and discard the rest. On the other hand, if we keep all of them and study their energy distribution, they actually approximate a Boltzmann distribution at some finite temperature: $P(E_i) = \frac {e^{-E_{i}/T}}{\sum _{j=1}^{M}{e^{-E_{j}/T}}}$. We can use this phenomenon for training certain machine learning models, as we will see later. In this notebook, we gain some more insight on preparing the thermal state by annealing and also by a variational circuit.

# Quantum annealing

Let us start by importing a handful of packages:

In [1]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import dimod
%matplotlib inline

We are interested in the thermal state of the classical Ising model. We create a random model over ten spins and we will sample a hundred states.

In [2]:
n_spins = 10
n_samples = 1000
h = {v: np.random.uniform(-2, 2) for v in range(n_spins)}
J = {}
for u, v in itertools.combinations(h, 2):
    if np.random.random() < .05:
        J[(u, v)] = np.random.uniform(-1, 1)
model = dimod.BinaryQuadraticModel(h, J, 0.0, dimod.SPIN)
sampler = dimod.SimulatedAnnealingSampler()        

Let's sample the energies at different temperatures. The `dimod` implementation of simulated annealing allows us to set an initial and final temperature for the annealing. If we set it to the same value, we mimic the effect of a finite temperature and we will have a wider range of configurations and energy levels in the samples. The next cell can take a while to execute.

In [ ]:
temperature_0 = 1
response = sampler.sample(model, beta_range=[1/temperature_0, 1/temperature_0], num_reads=n_samples)
energies_0 = [solution.energy for solution in response.data()]
temperature_1 = 10
response = sampler.sample(model, beta_range=[1/temperature_1, 1/temperature_1], num_reads=n_samples)
energies_1 = [solution.energy for solution in response.data()]
temperature_2 = 100
response = sampler.sample(model, beta_range=[1/temperature_2, 1/temperature_2], num_reads=n_samples)
energies_2 = [solution.energy for solution in response.data()]

We define a function to plot the resulting probability distribution.

In [ ]:
def plot_probabilities(energy_samples, temperatures):
    fig, ax = plt.subplots()
    for i, (energies, T) in enumerate(zip(energy_samples, temperatures)):
        probabilities = np.exp(-np.array(sorted(energies))/T)
        Z = probabilities.sum()
        probabilities /= Z
        ax.plot(energies, probabilities, linewidth=3, label = "$T_" + str(i+1)+"$")
    minimum_energy = min([min(energies) for energies in energy_samples])
    maximum_energy = max([max(energies) for energies in energy_samples])
    ax.set_xlim(minimum_energy, maximum_energy)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('Energy')
    ax.set_ylabel('Probability')
    ax.legend()
    plt.show()

plot_probabilities([energies_0, energies_1, energies_2], 
                   [temperature_0, temperature_1, temperature_2])

Just as we saw in the notebook on evolution in open systems, the distribution flattens out at a high temperature ($T_3$). On the other hand, the energy is peaked for a low temperature, and we do not even have samples for high-energy states.

# Quantum Approximate Thermalization

There are many results for preparing a thermal state on a gate-model quantum computer, but most of them need a large-scale device. More recently, a protocol for approximating thermalization was developed using shallow circuits. The idea is that if we knew that the thermal state was a qubit $\psi$ (which means $\rho=|\psi \rangle \langle \psi |$), we could apply QAOA, starting from the ground state. The trick is to purify $\rho$ on a larger Hilbert space. If we call $\mathcal{H_1}$ our current Hilbert space, purifying a density matrix $\rho$ consists of finding a second Hilbert space $\mathcal{H_2}$ such that there exists $| \psi \rangle \in \mathcal{H_1} \otimes \mathcal{H_2}$ such that $\rho = \textrm{Tr}_{\mathcal{H_2}} \left( |\psi \rangle \langle \psi | \right)$, where $\textrm{Tr}_{\mathcal{H_2}}$ is the partial trace taken over the second Hilbert space -- in essence, we are marginalizing the probability distribution. It can be shown that $| \psi \rangle =\sqrt{2 \cosh \beta} \sum_{z \in {-1,1}} e^{-\beta z} |z \rangle_{\mathcal{H_1}} \otimes | z \rangle_{\mathcal{H_2}}$ purifies $\rho=e^{-\beta H_m}$ [[1](#1)]. This state can be built with a circuit composed uniquely of RX gates and CNOT gates

In [3]:
import numpy as np

from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit.wrapper import execute as q_execute
from qiskit.tools.qi.pauli import Pauli
from qiskit.tools.apps.optimization import eval_hamiltonian 
from qiskit_aqua.operator import Operator
from qiskit_aqua import get_initial_state_instance, get_algorithm_instance, get_optimizer_instance

from qiskit import Aer
from qiskit.tools.qi.qi import state_fidelity

from scipy.optimize import minimize
from functools import reduce
import itertools

np.set_printoptions(precision=2)


In [4]:
n_qubits = 3
n_system = n_qubits * 2 # for purification

In [5]:
beta = 1 # inverse temperature

In [6]:
weights = np.random.uniform(-1,1, size=(n_qubits, n_qubits))

In [7]:
p = 1 # QAOA parameter

## Preparing the hamiltonians

In [8]:
def pauli_z(qubit, coeff):
    eye = np.eye((n_system))
    return Operator([[coeff, Pauli(eye[qubit], np.zeros(n_system))]])

def pauli_x(qubit, coeff):
    eye = np.eye((n_system))
    return Operator([[1, Pauli(np.zeros(n_system), eye[qubit])]])

def product_pauli_z(q1, q2, coeff):
    eye = np.eye((n_system))
    return Operator([[coeff, Pauli(eye[q1], np.zeros(n_system)) * Pauli(eye[q2], np.zeros(n_system))]])

In [9]:
def ising_hamiltonian(weights):
    H = reduce(lambda x,y:x+y,
            [product_pauli_z(i,j, -weights[i,j])
             for (i,j) in itertools.product(range(n_qubits), range(n_qubits))])
    H.to_matrix()
    return H

In [10]:
Hc = ising_hamiltonian(weights)

## Initial state preparation
We prepare the intial state $|\psi_0 \rangle = \sqrt{2 cosh(\beta)} \sum_{z \in {1, -1}} e^{-\beta z} | z \rangle_S \otimes | z \rangle_E$, with $E$ a temporary space used for purification purpose. It can be shown that tracing out this state over $E$ reproduces the state $\rho \propto e^{-\beta H_m} $

In [11]:
qr = QuantumRegister(n_qubits * 2)
cr = ClassicalRegister(n_qubits)
backend = Aer.get_backend('qasm_simulator')

In [12]:
circuit_init = QuantumCircuit(qr)

alpha = 2 * np.arctan(np.exp(- beta / 2))
for i in range(n_qubits):
    circuit_init.rx(alpha, qr[n_qubits+i])
    circuit_init.cx(qr[n_qubits+i], qr[i])

## Useful functions

In [13]:
from qiskit_aqua.algorithms.adaptive.qaoa import QAOA
from qiskit_aqua.algorithms.adaptive.qaoa.varform import QAOAVarForm
from qiskit import execute
from qiskit import Aer

The class `QAOAVarForm` takes an initial state as argument, which should be an object containing a method `construct_circuit` that builds and returns the initial state

In [14]:
class InitialState:
    def __init__(self, beta, n_qubits):
        self.beta = beta
        self.n_qubits = n_qubits
    
    def construct_circuit(self, mode, qr):
        circuit_init = QuantumCircuit(qr)

        alpha = 2 * np.arctan(np.exp(- self.beta / 2))
        for i in range(n_qubits):
            circuit_init.rx(alpha, qr[n_qubits+i])
            circuit_init.cx(qr[n_qubits+i], qr[i])
            
        return circuit_init

We need to overload the original QAOA class to be able pass it an initial state as a parameter

In [15]:
class MyQAOA(QAOA):
    def init_args(self, operator, operator_mode, p, optimizer, init_state=None, opt_init_point=None, aux_operators=[]):
        """
        Args:
            operator (Operator): Qubit operator
            operator_mode (str): operator mode, used for eval of operator
            p (int) : the integer parameter p as specified in https://arxiv.org/abs/1411.4028
            optimizer (Optimizer) : the classical optimization algorithm.
            opt_init_point (str) : optimizer initial point.
        """
        var_form = QAOAVarForm(operator, p, init_state)
        super(QAOA, self).init_args(operator, operator_mode, var_form, optimizer, opt_init_point=opt_init_point)

In [16]:
class MyQAOAVarForm(QAOAVarForm):
    def construct_circuit(self, angles, quantum_register, classical_register):
        if not len(angles) == self.num_parameters:
            raise ValueError('Incorrect number of angles: expecting {}, but {} given.'.format(
                self.num_parameters, len(angles)
            ))
        q = quantum_register
        circuit = QuantumCircuit(quantum_register, classical_register)
        if self._initial_state:
            circuit += self._initial_state.construct_circuit('circuit', q)
        else:
            circuit.u2(0, np.pi, q)
        for idx in range(self._p):
            beta, gamma = angles[idx], angles[idx + self._p]
            circuit += self._cost_operator.evolve(None, gamma, 'circuit', 1, quantum_registers=q)
            circuit += self._mixer_operator.evolve(None, beta, 'circuit', 1, quantum_registers=q)
        return circuit

In [17]:
initial_state = InitialState(beta, n_qubits)

In [18]:
def get_thermal_state(weights, p):
    Hc = ising_hamiltonian(weights)
    print("Begin QAOA...")
    
    qaoa = MyQAOA()
    qaoa.setup_quantum_backend(shots=100)
    optimizer = get_optimizer_instance('COBYLA')
    qaoa.init_args(Hc, "matrix", p, optimizer, initial_state)
    result = qaoa.run()
    
    print("Results of QAOA", result)
    
    return result

In [19]:
result = get_thermal_state(weights, 1)

Begin QAOA...
Results of QAOA {'eigvals': array([0.07]), 'opt_params': array([1.00e-04, 1.12e+00]), 'eigvecs': array([[ 6.25e-01+3.75e-04j, -3.65e-06-2.48e-05j,  2.77e-05-3.65e-05j,
        -1.91e-09-1.31e-09j,  3.35e-05-8.03e-05j, -5.94e-09-4.56e-09j,
        -5.21e-09-8.40e-09j, -9.92e-13+5.14e-13j, -3.65e-06-2.48e-05j,
        -3.77e-01-3.69e-02j, -1.91e-09-1.31e-09j, -1.45e-05+1.74e-05j,
        -5.94e-09-4.56e-09j,  1.21e-05+2.10e-05j, -9.92e-13+5.14e-13j,
         7.00e-11-1.52e-09j,  2.77e-05-3.65e-05j, -1.91e-09-1.31e-09j,
        -2.60e-01+2.76e-01j,  1.68e-05+5.68e-06j, -5.21e-09-8.40e-09j,
        -9.92e-13+5.14e-13j,  5.05e-05+2.82e-05j,  7.93e-10-5.36e-09j,
        -1.91e-09-1.31e-09j, -1.45e-05+1.74e-05j,  1.68e-05+5.68e-06j,
         2.03e-01-1.08e-01j, -9.92e-13+5.14e-13j,  7.00e-11-1.52e-09j,
         7.93e-10-5.36e-09j,  3.13e-06-2.03e-05j,  3.35e-05-8.03e-05j,
        -5.94e-09-4.56e-09j, -5.21e-09-8.40e-09j, -9.92e-13+5.14e-13j,
         1.78e-01+3.35e-01j,  4.92e-0

In [20]:
var_form = MyQAOAVarForm(Hc, p, initial_state)

In [21]:
thermal_state = var_form.construct_circuit([0.1,2.5], qr, cr)

In [22]:
thermal_state.get_qregs()

OrderedDict([('q0', QuantumRegister(6, 'q0'))])

In [ ]:
for i in range(n_qubits):
    thermal_state.measure(qr[i], cr[i])

In [ ]:
job = execute(thermal_state, backend, shots=100)
results = job.result().get_counts(thermal_state)
results

## Plot the energy histogram

In [ ]:
def get_energy(spin_configuration):
    x = spin_configuration.reshape(-1, 1)
    return spin_configuration.T.dot(weights).dot(spin_configuration)

In [ ]:
list_spin_configs = np.array(np.concatenate([[list(spin_config)] * results[spin_config] for spin_config in results]), dtype="int")

In [ ]:
list_energy = [get_energy(str_to_array(spin_config)) for spin_config in list_spin_configs]

In [ ]:
def get_probability(energy, Z=1):
    return np.exp(- beta * energy) / Z

In [ ]:
list_proba = [get_probability(energy) for energy in list_energy]

In [ ]:
plt.hist(list_energy)

# References

[1] Verdon, G., Broughton, M., Biamonte, J. (2017) [A quantum algorithm to train neural networks using low-depth circuits](https://arxiv.org/abs/1712.05304). *arXiv:1712.05304*. <a id='1'></a>